In [1]:
import os

In [2]:
%pwd

'd:\\recommendation-engine\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\recommendation-engine'

In [5]:
import pandas as pd

In [6]:
bookings = pd.read_csv("artifacts/data_ingestion/generated_data/bookings.csv")
comments = pd.read_csv("artifacts/data_ingestion/generated_data/comments.csv")
likes = pd.read_csv("artifacts/data_ingestion/generated_data/likes.csv")
bookings.head()
comments.head()
likes.head()

,user_id,liked_event_id,liked_event_organizer_id,like_id
0,c238b607-febe-414c-9659-13eaf32995be,a5f7b368-33b4-46f8-aaf3-c86046ce6090,28b59946-eca4-49d0-aff5-d47cd7f84099,1fcb99e6-e8a5-4995-9ce2-f3612e878ccd
1,f563d07d-c4b8-4594-bf43-287d3c97042c,f36848b3-37cd-4819-9803-8655375ed6a5,6dbcb86d-f273-4a68-aaa7-ab2eb7350841,94698481-2505-4fd5-8aa1-5193c4b3cb49
2,118c138f-9238-468e-9af7-4d370f843d55,3c14a12b-acf0-4497-92b1-c8aa227ce806,d9cdb0ec-869d-4368-bad3-04827128de42,8c0eb07d-213f-4093-9806-701b422cd6cf
3,49c94e76-8b59-498a-b6fc-a80ebd365716,11239cdc-f026-4e31-be0b-fdacfa4a08e9,479dbf00-a94c-4f57-b490-e111d48d2c79,6d31d721-186d-4eb3-b195-762ffff899eb
4,992d2daa-66a3-4da5-9095-1dd085735b25,92cefcee-b508-4218-a3f5-bbabe2604a25,ad72af03-e285-46d5-baa4-dc6d7dfa98bf,7b48bc9b-7c2e-466d-b3e8-8de0139127df


In [7]:
bookings.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   user_id                    150000 non-null  object
 1   booked_event_id            150000 non-null  object
 2   booked_event_organizer_id  150000 non-null  object
 3   booking_id                 150000 non-null  object
dtypes: object(4)
memory usage: 4.6+ MB


In [8]:
comments.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245397 entries, 0 to 245396
Data columns (total 4 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   user_id                       245397 non-null  object
 1   commented_event_id            245397 non-null  object
 2   commented_event_organizer_id  245397 non-null  object
 3   comment_id                    245397 non-null  object
dtypes: object(4)
memory usage: 7.5+ MB


In [9]:
likes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145397 entries, 0 to 145396
Data columns (total 4 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   user_id                   145397 non-null  object
 1   liked_event_id            145397 non-null  object
 2   liked_event_organizer_id  145397 non-null  object
 3   like_id                   145397 non-null  object
dtypes: object(4)
memory usage: 4.4+ MB


In [10]:
# Standardize column names across all datasets
bookings = bookings.rename(columns={
    'booked_event_id': 'event_id',
    'booked_event_organizer_id': 'organizer_id',
    'booking_id': 'id'
})

comments = comments.rename(columns={
    'commented_event_id': 'event_id',
    'commented_event_organizer_id': 'organizer_id',
    'comment_id': 'id'
})

likes = likes.rename(columns={
    'liked_event_id': 'event_id',
    'liked_event_organizer_id': 'organizer_id',
    'like_id': 'id'
})

# Create source-specific DataFrames with indicator columns
bookings_df = bookings[['user_id', 'event_id', 'organizer_id']].copy()
bookings_df['is_booked'] = 1
bookings_df['num_bookings'] = 1

comments_df = comments[['user_id', 'event_id', 'organizer_id']].copy()
comments_df['is_commented'] = 1
comments_df['num_comments'] = 1

likes_df = likes[['user_id', 'event_id', 'organizer_id']].copy()
likes_df['is_liked'] = 1
likes_df['num_likes'] = 1

# Get all unique user-event pairs
unique_pairs = pd.concat([
    bookings[['user_id', 'event_id']],
    comments[['user_id', 'event_id']],
    likes[['user_id', 'event_id']]
]).drop_duplicates()

# Randomly select 100,000 unique pairs if we have more than that
if len(unique_pairs) > 100000:
    unique_pairs = unique_pairs.sample(n=100000, random_state=42)

# Get consistent organizer_id mapping (most frequent for each event)
organizer_mapping = pd.concat([
    bookings[['event_id', 'organizer_id']],
    comments[['event_id', 'organizer_id']],
    likes[['event_id', 'organizer_id']]
]).groupby('event_id')['organizer_id'].agg(lambda x: x.mode()[0]).to_dict()

# Merge all interaction types
result = unique_pairs.copy()

# Merge booking data
result = result.merge(
    bookings_df.groupby(['user_id', 'event_id']).agg({
        'is_booked': 'max',
        'num_bookings': 'sum',
        'organizer_id': 'first'
    }).reset_index(),
    on=['user_id', 'event_id'],
    how='left'
)

# Merge comment data
result = result.merge(
    comments_df.groupby(['user_id', 'event_id']).agg({
        'is_commented': 'max',
        'num_comments': 'sum',
        'organizer_id': 'first'
    }).reset_index(),
    on=['user_id', 'event_id'],
    how='left'
)

# Merge like data
result = result.merge(
    likes_df.groupby(['user_id', 'event_id']).agg({
        'is_liked': 'max',
        'num_likes': 'sum',
        'organizer_id': 'first'
    }).reset_index(),
    on=['user_id', 'event_id'],
    how='left'
)

# Fill NA values with 0 for indicator and count columns
for col in ['is_booked', 'num_bookings', 'is_commented', 'num_comments', 'is_liked', 'num_likes']:
    result[col] = result[col].fillna(0).astype(int)

# Consolidate organizer_id (take first non-NA value)
result['organizer_id'] = (result['organizer_id_x']
                         .fillna(result['organizer_id_y'])
                         .fillna(result['organizer_id']))

# If still missing, use our mapping
result['organizer_id'] = result['organizer_id'].fillna(result['event_id'].map(organizer_mapping))

# Select final columns
final_data = result[[
    'user_id', 
    'event_id', 
    'is_booked',
    'num_bookings',
    'is_liked',
    'num_likes',
    'is_commented', 
    'num_comments', 
    'organizer_id'
]]

# Ensure we have exactly 100,000 records (in case some merges added duplicates)
final_data = final_data.drop_duplicates().sample(n=min(100000, len(final_data)), random_state=42)

# Save the merged data
final_data.to_csv("artifacts/data_ingestion/generated_data/merged_interactions.csv", index=False)

print("Merged data saved successfully!")
print(f"Final dataset shape: {final_data.shape}")
print(final_data.head())

Merged data saved successfully!
Final dataset shape: (100000, 9)
                                    user_id  \
75721  a1bbccb0-d0b1-426c-9c1b-1eb97d8e8527   
80184  8a5a1dfa-e44e-470b-a992-8b13e0233b5c   
19864  ec976f33-db50-4215-aefd-5e04dcb0cbf7   
76699  76a1b14c-9ee3-4f16-9fc0-1ba0c208a092   
92991  fc7db133-add9-447c-a2b0-42d3d105be76   

                                   event_id  is_booked  num_bookings  \
75721  66773a59-49d3-4f79-8513-e7ec5a5102bb          0             0   
80184  0e6bf3fd-d26f-457a-9645-90403d610c31          1             2   
19864  e593730a-8006-445d-97f6-9a85ca942501          0             0   
76699  1ad2bbdc-b57a-45bf-9fb9-d3e8818153f6          0             0   
92991  664f2b30-0fb0-4c7d-a23c-1dd76b992d95          0             0   

       is_liked  num_likes  is_commented  num_comments  \
75721         1          1             0             0   
80184         1          2             1             2   
19864         0          0             1     